In [1]:
import pandas as pd
import torch
import torchvision.transforms as transforms
from torchvision.models import vit_l_16, ViT_L_16_Weights
from torch.utils.data import DataLoader 
from torchvision.datasets import ImageFolder 
import numpy as np
from sklearn.metrics import normalized_mutual_info_score
from tqdm import tqdm
from finch import FINCH
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment

def clustering_accuracy(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1

    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    acc = w[row_ind, col_ind].sum() / y_pred.size
    return acc

def extract_features(loader, model):
    features = []
    labels = []
    model.eval()
    with torch.no_grad():
        for images, targets in tqdm(loader, desc="Extracting features"):
            images = images.to(device)
            feat = model(images)
            print("Output shape:", feat.shape)
            features.append(feat.cpu().numpy())
            labels.append(targets.numpy())
    return np.concatenate(features), np.concatenate(labels)

results = []
all_image_info = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

for trial in range(5):

    unlabeled_transform = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])

    unlabeled_folder = fr"C:\Users\HP\novelty\split_datasets\trial_{trial}\42labeled_10unlabeled\unlabeled"

    unlabeled_dataset = ImageFolder(root=unlabeled_folder, transform=unlabeled_transform) 
    print(f"Total gambar dalam unlabeled dataset: {len(unlabeled_dataset)}  {len(unlabeled_dataset.classes)}")

    batch_size = 16

    unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)
    print(f"Total loader gambar dalam unlabeled dataset: {len(unlabeled_loader)}")
    
    model = vit_l_16(weights=ViT_L_16_Weights.IMAGENET1K_V1)
    model.heads = torch.nn.Identity()
    model = model.to(device)
    model.eval()
    
    unlabeled_features, unlabeled_true_labels = extract_features(unlabeled_loader, model)
    
    
    c, num_clust, req_c = FINCH(unlabeled_features, use_ann_above_samples=1000, verbose=True)
    finch_clusters = c[:,2]

    finch_nmi = normalized_mutual_info_score(unlabeled_true_labels, finch_clusters)
    finch_acc = clustering_accuracy(unlabeled_true_labels, finch_clusters)

    print(f"\nFINCH Clustering Performance:")
    print(f"Clustering Accuracy (ACC): {finch_acc:.4f}")
    print(f"Normalized Mutual Information (NMI): {finch_nmi:.4f}")
    print(f"Number of clusters found: {len(np.unique(finch_clusters))}")
    
    num_clusters = len(np.unique(unlabeled_true_labels))
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans_clusters = kmeans.fit_predict(unlabeled_features)

    # ----- Evaluasi -----
    kmeans_nmi = normalized_mutual_info_score(unlabeled_true_labels, kmeans_clusters)
    kmeans_acc = clustering_accuracy(unlabeled_true_labels, kmeans_clusters)

    print(f"\nK-Means Clustering Performance: {num_clusters} Clusters")
    print(f"Clustering Accuracy (ACC): {kmeans_acc:.4f}")
    print(f"Normalized Mutual Information (NMI): {kmeans_nmi:.4f}")
    print(f"Number of clusters found: {len(np.unique(kmeans_clusters))}")
    
    # Simpan hasil trial ini
    results.append({
        'trial': trial,
        'FINCH_ACC': finch_acc,
        'FINCH_NMI': finch_nmi,
        'FINCH_Num_Clusters': len(np.unique(finch_clusters)),
        'KMeans_ACC': kmeans_acc,
        'KMeans_NMI': kmeans_nmi,
        'KMeans_Num_Clusters': len(np.unique(kmeans_clusters)),
    })
    
    # Simpan hasil per gambar
    for i in range(len(unlabeled_dataset)):
        path, true_label = unlabeled_dataset.samples[i]
        image_info = {
            'trial': trial,
            'image_path': path,
            'true_label': true_label,
            'finch_cluster': int(finch_clusters[i]),
            'kmeans_cluster': int(kmeans_clusters[i])
        }
        all_image_info.append(image_info)

    # Simpan file CSV per trial
    df_trial_detail = pd.DataFrame(all_image_info[-len(unlabeled_dataset):])  # ambil data dari trial ini saja
    df_trial_detail.to_csv(f'vit_l_16_pretrained_42labeled_10unlabeled_trial_{trial}_image_clustering.csv', index=False)

# Konversi ke DataFrame dan simpan ke CSV
df_results = pd.DataFrame(results)
mean_values = df_results.select_dtypes(include=np.number).mean()

mean_row = pd.DataFrame({
    'trial': ['Average'],
    'FINCH_ACC': [mean_values['FINCH_ACC']],
    'FINCH_NMI': [mean_values['FINCH_NMI']],
    'FINCH_Num_Clusters': [mean_values['FINCH_Num_Clusters']],
    'KMeans_ACC': [mean_values['KMeans_ACC']],
    'KMeans_NMI': [mean_values['KMeans_NMI']],
    'KMeans_Num_Clusters': [mean_values['KMeans_Num_Clusters']]
})

df_results = pd.concat([df_results, mean_row], ignore_index=True)

df_results.to_csv('clustering_results_vit_l_16_pretrained_42labeled_10unlabeled_FINCH_KMEANS.csv', index=False)
print("Results saved to clustering_results_vit_l_16_pretrained_42labeled_10unlabeled_FINCH_KMEANS.csv")

Using device: cuda
Total gambar dalam unlabeled dataset: 1000  10
Total loader gambar dalam unlabeled dataset: 63


Extracting features:   0%|          | 0/63 [00:00<?, ?it/s]

Output shape: torch.Size([16, 1024])


Extracting features:   2%|▏         | 1/63 [00:00<00:37,  1.68it/s]

Output shape: torch.Size([16, 1024])


...

Output shape: torch.Size([16, 1024])


Extracting features:  98%|█████████▊| 62/63 [00:29<00:00,  2.13it/s]

Output shape: torch.Size([8, 1024])


Extracting features: 100%|██████████| 63/63 [00:29<00:00,  2.12it/s]

Partition 0: 227 clusters
Partition 1: 47 clusters
Partition 2: 14 clusters
Partition 3: 4 clusters

FINCH Clustering Performance:
Clustering Accuracy (ACC): 0.8240
Normalized Mutual Information (NMI): 0.8930
Number of clusters found: 14

K-Means Clustering Performance: 10 Clusters
Clustering Accuracy (ACC): 0.6310
Normalized Mutual Information (NMI): 0.7591
Number of clusters found: 10
Results saved to clustering_results_vit_l_16_pretrained_42labeled_10unlabeled_FINCH_KMEANS.csv
